In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark=SparkSession.builder.appName("nyc_taxi").config("spark.local.dir", "/home/tb24/projects/Datalake-project").getOrCreate()

23/06/21 22:12:45 WARN Utils: Your hostname, Legion resolves to a loopback address: 127.0.1.1; using 172.21.29.24 instead (on interface eth0)
23/06/21 22:12:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/21 22:12:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/06/21 22:12:46 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
23/06/21 22:12:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
df = spark.read.parquet("/home/tb24/projects/Datalake-project/data/yellow_tripdata_2023-03.parquet")
df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [4]:
df.createOrReplaceGlobalTempView("dfTable")

In [5]:
from pyspark.sql.functions import avg
# Average fee of a taxi ride
df.selectExpr("avg(total_amount)").show(2)
# Average trip Distance
df.select(avg("trip_distance")).show(2)
# Average tips, tolls and fare and congestion surcharge
df.select(avg("tip_amount").alias("average_tip_amount"),
          avg("tolls_amount").alias('average_toll_amount'),
          avg("fare_amount").alias('average_fare'),
          avg("congestion_surcharge").alias('average_congestion_surcharge')
).show(2)

+------------------+
| avg(total_amount)|
+------------------+
|27.803426281277332|
+------------------+

+------------------+
|avg(trip_distance)|
+------------------+
| 3.903870542216571|
+------------------+



+------------------+-------------------+-----------------+----------------------------+
|average_tip_amount|average_toll_amount|     average_fare|average_congestion_surcharge|
+------------------+-------------------+-----------------+----------------------------+
|3.4952374575695115| 0.5670058928868822|18.90844797498149|           2.275175678279642|
+------------------+-------------------+-----------------+----------------------------+



In [6]:
from pyspark.sql.functions import min, max, count_distinct, col
# Biggest number of Passengers
df.select(max('passenger_count')).show(2)
# Biggest fare for a ride and its trip distance
df.groupby('trip_distance').agg(max('fare_amount')).show(2)

+--------------------+
|max(passenger_count)|
+--------------------+
|                   9|
+--------------------+



+-------------+----------------+
|trip_distance|max(fare_amount)|
+-------------+----------------+
|         9.13|            70.0|
|         14.9|           118.2|
+-------------+----------------+
only showing top 2 rows



In [18]:
from pyspark.sql.functions import desc, asc, expr
df.select(col("trip_distance")).orderBy(col("trip_distance").desc()).show(5)
df.select(max("trip_distance")).show(1)

# Outlier - big miles small prices what the hell
df.select(col('total_amount'), col('trip_distance'), col('Store_and_fwd_flag'))\
    .where(col('trip_distance') == 216986.96).show(2)

+-------------+
|trip_distance|
+-------------+
|    216986.96|
|    157733.41|
|    144561.23|
|    103942.41|
|      70457.9|
+-------------+
only showing top 5 rows

+------------------+
|max(trip_distance)|
+------------------+
|         216986.96|
+------------------+

+------------+-------------+------------------+
|total_amount|trip_distance|Store_and_fwd_flag|
+------------+-------------+------------------+
|       39.25|    216986.96|              null|
+------------+-------------+------------------+

